![Transfer Learning](./transfer_learning.jpg "Transfer Learning")

![VGG16](./vgg16.webp "VGG16")

![VGG16](./vgg16_2.webp "VGG16")

In [1]:
import torch
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from time import time

import dataset
from finetune import ModifiedVGG16Model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def evaluate_model(model_, test_loader_, device_, threshold=None):
    model_.eval()
    all_preds = []
    all_labels = []
    batch_times = []
    total_inference_time = 0.
    i = 0

    with torch.no_grad():
        for batch, labels in test_loader_:
            if threshold is not None:
                if i >= threshold:
                    break

            batch, labels = batch.to(device_), labels.to(device_)

            batch_start = time()
            outputs = model_(batch)
            batch_time = time() - batch_start
            batch_times.append(batch_time)
            total_inference_time += batch_time

            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            if threshold is not None:
                i += batch.shape[0]

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )

    avg_batch_time = np.mean(batch_times) * 1000  # ms

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "avg_batch_time": avg_batch_time,
    }

In [4]:
def print_result(cache):
    print(f"Accuracy: {np.mean(cache["accuracy"]):.4f}")
    print(f"Precision: {np.mean(cache["precision"]):.4f}")
    print(f"Recall: {np.mean(cache["recall"]):.4f}")
    print(f"F1-Score: {np.mean(cache["f1"]):.4f}")
    print(f"Average batch time: {np.mean(cache["avg_batch_time"]):.4f} ms")

In [5]:
def compute_mean(model_, test_loader_, device_, num_iter):
    cache = {
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1": [],
        "avg_batch_time": []
    }

    for _ in range(num_iter):
        temp = evaluate_model(model_, test_loader_, device_)
        cache["accuracy"].append(temp["accuracy"])
        cache["precision"].append(temp["precision"])
        cache["recall"].append(temp["recall"])
        cache["f1"].append(temp["f1"])
        cache["avg_batch_time"].append(temp["avg_batch_time"])

    # print(cache["avg_batch_time"])
    # print(sum(cache["avg_batch_time"]))
    # print(sum(cache["avg_batch_time"]) / len(cache["avg_batch_time"]))

    print_result(cache)

### VGG16 GPU

In [ ]:
test_path = "/home/washindeiru/studia/sem_8/ssn/sem/pytorch-pruning/data/animals10/test"

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)

compute_mean(model, test_loader, device, 10)

Accuracy: 0.9521
Precision: 0.9531
Recall: 0.9521
F1-Score: 0.9522
Average batch time: 1.0880 ms


### VGG16 pruned GPU

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55", map_location=device, weights_only=False)
model = model.to(device)

compute_mean(model, test_loader, device, 10)

Accuracy: 0.9343
Precision: 0.9372
Recall: 0.9343
F1-Score: 0.9341
Average batch time: 1.2586 ms


### VGG16 CPU

In [8]:
device = torch.device("cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)
result = evaluate_model(model, test_loader, device, 100)
print_result(result)

Accuracy: 0.9688
Precision: 0.9681
Recall: 0.9658
F1-Score: 0.9660
Average batch time: 2877.3497 ms


### VGG16 pruned CPU

In [9]:
device = torch.device("cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55", map_location=device, weights_only=False)
result = evaluate_model(model, test_loader, device, 100)
print_result(result)

Accuracy: 0.9141
Precision: 0.9269
Recall: 0.9157
F1-Score: 0.9166
Average batch time: 1182.3679 ms


## Liczba parametrów

VGG16

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"VG16 łączna liczba parameterów: {total_params}")

total_conv_params = sum(p.numel() for p in model.features.parameters())
print(f"VG16 część konwolucyjna, liczba parameterów: {total_conv_params}")

total_class_params = sum(p.numel() for p in model.classifier.parameters())
print(f"VG16 część klasyfikatora, liczba parameterów: {total_class_params}")

VG16 łączna liczba parameterów: 134301514
VG16 część konwolucyjna, liczba parameterów: 14714688
VG16 część klasyfikatora, liczba parameterów: 119586826


VGG16 pruned

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55", map_location=device, weights_only=False)
model = model.to(device)

total_params_pruned = sum(p.numel() for p in model.parameters())
print(f"VG16 łączna liczba parameterów: {total_params_pruned}")

total_conv_params_pruned = sum(p.numel() for p in model.features.parameters())
print(f"VG16 część konwolucyjna, liczba parameterów: {total_conv_params_pruned}")

total_class_params_pruned = sum(p.numel() for p in model.classifier.parameters())
print(f"VG16 część klasyfikatora, liczba parameterów: {total_class_params_pruned}")

VG16 łączna liczba parameterów: 38664702
VG16 część konwolucyjna, liczba parameterów: 2169332
VG16 część klasyfikatora, liczba parameterów: 36495370


| Model         | Łączna liczba parametrów     | Część konwolucyjna, liczba parametrów     | Część klasyfikatora, liczba parameterów     | Rozmiar pliku |
|---------------|------------------------------|-------------------------------------------|---------------------------------------------|---------------|
| VGG 16        | 134 301 514                  | 14 714 688                                | 119 586 826                                 | 537.2MB       |
| VGG 16 pruned | 38 664 702                   | 2 169 332                                 | 36 495 370                                  | 154.7MB       |

### Liczba usuniętych filtrów

In [23]:
iter_1 = {21: 57, 17: 61, 28: 157, 26: 68, 14: 16, 19: 49, 12: 12, 24: 56, 5: 8, 10: 20, 7: 3, 2: 2, 0: 3}
iter_2 = {28: 113, 12: 23, 21: 61, 26: 96, 17: 48, 19: 50, 24: 73, 0: 4, 14: 13, 10: 15, 7: 12, 5: 3, 2: 1}
iter_3 = {17: 53, 24: 77, 12: 20, 21: 81, 19: 64, 26: 79, 28: 55, 14: 31, 10: 22, 5: 11, 7: 7, 2: 6, 0: 6}
iter_4 = {7: 10, 17: 77, 19: 65, 10: 29, 2: 6, 28: 47, 26: 65, 21: 69, 24: 72, 12: 27, 14: 24, 0: 5, 5: 16}
iter_5 = {7: 21, 14: 39, 24: 42, 2: 6, 19: 80, 17: 66, 12: 39, 5: 14, 21: 56, 26: 68, 0: 4, 10: 35, 28: 42}

removed_filters = {}

for temp in (iter_1, iter_2, iter_3, iter_4, iter_5):
    for k,v in temp.items():
        if k in removed_filters:
            removed_filters[k] += v
        else:
            removed_filters[k] = v

keys = sorted(removed_filters.keys())
for key in keys:
    print(f"{key}: {removed_filters[key]}")

0: 22
2: 21
5: 52
7: 53
10: 121
12: 121
14: 123
17: 305
19: 308
21: 324
24: 320
26: 376
28: 414


| **Numer warstwy**             | 0  | 2  | 5  | 7  | 10  | 12  | 14  | 17  | 19  | 21  | 24  | 26  | 28  |
|-------------------------------|----|----|----|----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| **Liczba usuniętych filtrów** | 22 | 21 | 52 | 53 | 121 | 121 | 123 | 305 | 308 | 324 | 320 | 376 | 414 |